In [1]:
from lfw import gen_all_pic_labels
import pandas as pd
import numpy as np
from deepface import DeepFace
from deepface.commons import functions
import arcface_align
import matplotlib.pyplot as plt
import random
import cv2
import os

In [2]:
all_pics = gen_all_pic_labels('./lfw_funneled/')

In [3]:
pics,names,genders = gen_all_pic_labels('./lfw_funneled/')
#prepare gender
df = pd.DataFrame({'files':pics,'name':names,'gender':genders})

In [4]:
namecount = pd.DataFrame(df.name.value_counts())

In [5]:
count = df.name.value_counts()

In [6]:
filter_names = count[count > 1].keys().to_list()

In [7]:
filter_names

['George_W_Bush',
 'Colin_Powell',
 'Tony_Blair',
 'Donald_Rumsfeld',
 'Gerhard_Schroeder',
 'Ariel_Sharon',
 'Hugo_Chavez',
 'Junichiro_Koizumi',
 'Jean_Chretien',
 'John_Ashcroft',
 'Jacques_Chirac',
 'Serena_Williams',
 'Vladimir_Putin',
 'Luiz_Inacio_Lula_da_Silva',
 'Gloria_Macapagal_Arroyo',
 'Arnold_Schwarzenegger',
 'Jennifer_Capriati',
 'Laura_Bush',
 'Lleyton_Hewitt',
 'Hans_Blix',
 'Alejandro_Toledo',
 'Nestor_Kirchner',
 'Andre_Agassi',
 'Alvaro_Uribe',
 'Megawati_Sukarnoputri',
 'Silvio_Berlusconi',
 'Tom_Ridge',
 'Vicente_Fox',
 'Roh_Moo-hyun',
 'Kofi_Annan',
 'John_Negroponte',
 'David_Beckham',
 'Recep_Tayyip_Erdogan',
 'Guillermo_Coria',
 'Mahmoud_Abbas',
 'Bill_Clinton',
 'Juan_Carlos_Ferrero',
 'Jack_Straw',
 'Ricardo_Lagos',
 'Rudolph_Giuliani',
 'Gray_Davis',
 'Tom_Daschle',
 'Jeremy_Greenstock',
 'Winona_Ryder',
 'Atal_Bihari_Vajpayee',
 'Saddam_Hussein',
 'Tiger_Woods',
 'Jose_Maria_Aznar',
 'Lindsay_Davenport',
 'Naomi_Watts',
 'George_Robertson',
 'Hamid_Karzai

## There are 1680 people with more than one face image

In [8]:
df_filtered = df[df['name'].isin(filter_names)]

In [9]:
random.shuffle(filter_names)

In [10]:
def consine_sim(x,y):
    return 1 - np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

In [13]:
def consine_sim(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

## Campare the embedding for same person

In [14]:
dep_stds = []
dep_means = []
arc_stds = []
arc_means = []
random.shuffle(filter_names)
arc_dis = []
dep_dis = []

arc_sim = []
dep_sim = []
for num in range(0,200):
    name = filter_names[num]
    df_name = df[df['name'] == name]
    filenames = list(df_name['files'])
    emb_arc_align = np.zeros((len(filenames),512))
    emb_dep_align = np.zeros((len(filenames),512))
    for i in range(len(filenames)):
        file = filenames[i]
        try:
            emb_dep_align[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass
        try:
            arc_align = arcface_align.get_face_align(file)
            emb_arc_align[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass

    cons_sim_dep_align = []
    cons_sim_arc_align = []
    dep_no_zero = []
    arc_no_zero = []
    for i in range(len(filenames)):
        for j in range(i+1,len(filenames)):
            if(np.all(emb_arc_align[i,:] == 0) or np.all(emb_arc_align[j,:] == 0)):
                continue
            if(np.all(emb_dep_align[i,:] == 0) or np.all(emb_dep_align[j,:] == 0)):
                continue
            dep_no_zero.append(emb_dep_align[i,:])
            dep_no_zero.append(emb_dep_align[j,:])
            arc_no_zero.append(emb_arc_align[i,:])
            arc_no_zero.append(emb_arc_align[j,:])
            cons_sim_arc_align.append(consine_sim(emb_arc_align[i,:],emb_arc_align[j,:]))
            cons_sim_dep_align.append(consine_sim(emb_dep_align[i,:],emb_dep_align[j,:]))
    if(len(cons_sim_arc_align) == 0 or len(cons_sim_dep_align)==0):
        continue
    arc_dis.append(np.mean(cons_sim_arc_align))
    dep_dis.append(np.mean(cons_sim_dep_align))
    arc_stds.append(np.std(cons_sim_arc_align))
    dep_stds.append(np.std(cons_sim_dep_align))
print('arc mean:',np.mean(arc_dis))
print('dep mean:',np.mean(dep_dis))
print('arc std:',np.mean(arc_stds))
print('dep std:',np.mean(dep_stds))


In [30]:
total_compare = 0
arc_lt_deep = 0
#shuffle the names
random.shuffle(filter_names)
#total number of names
for num in range(0,50):
    name = filter_names[num]
    df_name = df[df['name'] == name]
    #get the filenames for this person
    filenames = list(df_name['files'])
    #array to save embeddings
    emb_arc_align = np.zeros((len(filenames),512))
    emb_dep_align = np.zeros((len(filenames),512))
    for i in range(len(filenames)):
        file = filenames[i]
        try:
            emb_dep_align[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass
        try:
            arc_align = arcface_align.get_face_align(file)
            emb_arc_align[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass

    #compare
    for i in range(len(filenames)):
        for j in range(i+1,len(filenames)):
            if(np.all(emb_arc_align[i,:] == 0) or np.all(emb_arc_align[j,:] == 0)):
                break
            if(np.all(emb_dep_align[i,:] == 0) or np.all(emb_dep_align[j,:] == 0)):
                break
            total_compare +=1
            arc_sim  = consine_sim(emb_arc_align[i,:],emb_arc_align[j,:])
            dep_sim = consine_sim(emb_dep_align[i,:],emb_dep_align[j,:])
            if(arc_sim<dep_sim):
                arc_lt_deep +=1

print('Probability of arc sim less than deep sim',arc_lt_deep/total_compare)

    

Probability of arc sim less than deep sim 0.48095238095238096


In [28]:
cons_sim_arc_align

[0.3797528206254389, 0.5052423293224397, 0.5038667146720051]

## Compare the embedding between different person


In [16]:
comp_pairs = 50
total_compare = 0
arc_more = 0
for num in range(comp_pairs):
    persona = ''
    personb = ''

    while(persona == personb):
        persona = filter_names[random.randint(0,len(filter_names))]
        personb = filter_names[random.randint(0,len(filter_names))]
    df_p_a = df[df['name'] == persona]
    #get the filenames for this person
    files_a = list(df_p_a['files'])

    df_p_b = df[df['name'] == personb]
    #get the filenames for this person
    files_b = list(df_p_b['files'])
    # print(files_a,files_b)
    emb_arc_a = np.zeros((len(files_a),512))
    emb_arc_b = np.zeros((len(files_b),512))
    emb_dep_a = np.zeros((len(files_a),512))
    emb_dep_b = np.zeros((len(files_b),512))
    
    for i in range(len(files_a)):
        file = files_a[i]
        try:
            emb_dep_a[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass
        try:
            arc_align = arcface_align.get_face_align(file)
            emb_arc_a[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=False,enforce_detection=True)
        except:
            pass
    for i in range(len(files_b)):
        file = files_b[i]
        try:
            emb_dep_b[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass
        try:
            arc_align = arcface_align.get_face_align(file)
            emb_arc_b[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=False,enforce_detection=True)
        except:
            pass

    for i in range(len(files_a)):
        for j in range(len(files_b)):
            if(np.all(emb_arc_a[i,:] == 0) or np.all(emb_dep_a[i,:] == 0) or np.all(emb_dep_b[j,:] == 0) or np.all(emb_arc_b[j,:] == 0) ):
                continue
            dis_arc = consine_sim(emb_arc_a[i,:],emb_arc_b[j,:])
            dis_dep = consine_sim(emb_dep_a[i,:],emb_dep_b[j,:])
            total_compare +=1
            if(dis_arc > dis_dep):
                arc_more +=1


print(total_compare,arc_more,arc_more/total_compare)     



[ WARN:0@506.039] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./lfw_funneled/Zhu_Rongji/Zhu_Rongji_0004.jpg'): can't open/read file: check file path/integrity
[ WARN:0@506.175] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./lfw_funneled/Zhu_Rongji/Zhu_Rongji_0008.jpg'): can't open/read file: check file path/integrity
[ WARN:0@506.313] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./lfw_funneled/Zhu_Rongji/Zhu_Rongji_0006.jpg'): can't open/read file: check file path/integrity
[ WARN:0@506.446] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./lfw_funneled/Zhu_Rongji/Zhu_Rongji_0001.jpg'): can't open/read file: check file path/integrity
[ WARN:0@506.583] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('./lfw_funneled/Zhu_Rongji/Zhu_Rongji_0002.jpg'): can't open/read file: check file path/integrity
[ WARN:0@506.711] global 

In [12]:
for batch in range(10):
    comp_pairs = 50
    total_compare = 0
    arc_more = 0
    for num in range(comp_pairs):
        persona = ''
        personb = ''

        while(persona == personb):
            persona = filter_names[random.randint(0,len(filter_names))]
            personb = filter_names[random.randint(0,len(filter_names))]
        df_p_a = df[df['name'] == persona]
        #get the filenames for this person
        files_a = list(df_p_a['files'])

        df_p_b = df[df['name'] == personb]
        #get the filenames for this person
        files_b = list(df_p_b['files'])
        # print(files_a,files_b)
        emb_arc_a = np.zeros((len(files_a),512))
        emb_arc_b = np.zeros((len(files_b),512))
        emb_dep_a = np.zeros((len(files_a),512))
        emb_dep_b = np.zeros((len(files_b),512))
        
        for i in range(len(files_a)):
            file = files_a[i]
            try:
                emb_dep_a[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
            except:
                pass
            try:
                arc_align = arcface_align.get_face_align(file)
                emb_arc_a[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=False,enforce_detection=True)
            except:
                pass
        for i in range(len(files_b)):
            file = files_b[i]
            try:
                emb_dep_b[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
            except:
                pass
            try:
                arc_align = arcface_align.get_face_align(file)
                emb_arc_b[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=False,enforce_detection=True)
            except:
                pass

        for i in range(len(files_a)):
            for j in range(len(files_b)):
                if(np.all(emb_arc_a[i,:] == 0) or np.all(emb_dep_a[i,:] == 0) or np.all(emb_dep_b[j,:] == 0) or np.all(emb_arc_b[j,:] == 0) ):
                    continue
                dis_arc = consine_sim(emb_arc_a[i,:],emb_arc_b[j,:])
                dis_dep = consine_sim(emb_dep_a[i,:],emb_dep_b[j,:])
                total_compare +=1
                if(dis_arc > dis_dep):
                    arc_more +=1


    print(total_compare,arc_more,arc_more/total_compare)     



2046 1087 0.5312805474095796
462 217 0.4696969696969697
1149 731 0.6362053959965187
1078 492 0.45640074211502785
865 508 0.5872832369942197
1028 770 0.7490272373540856


In [1]:
os.getcwd()

'/home/mafffia/Desktop/Face-recognition-related'

In [11]:
comp_pairs = 50
total_compare = 0
arc_more = 0
eps = 0.1
for num in range(comp_pairs):
    persona = ''
    personb = ''

    while(persona == personb):
        persona = filter_names[random.randint(0,len(filter_names))]
        personb = filter_names[random.randint(0,len(filter_names))]
    df_p_a = df[df['name'] == persona]
    #get the filenames for this person
    files_a = list(df_p_a['files'])

    df_p_b = df[df['name'] == personb]
    #get the filenames for this person
    files_b = list(df_p_b['files'])
    # print(files_a,files_b)
    emb_arc_a = np.zeros((len(files_a),512))
    emb_arc_b = np.zeros((len(files_b),512))
    emb_dep_a = np.zeros((len(files_a),512))
    emb_dep_b = np.zeros((len(files_b),512))
    
    for i in range(len(files_a)):
        file = files_a[i]
        try:
            emb_dep_a[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass
        try:
            arc_align = arcface_align.get_face_align(file)
            emb_arc_a[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=False,enforce_detection=True)
        except:
            pass
    for i in range(len(files_b)):
        file = files_b[i]
        try:
            emb_dep_b[i,:] = DeepFace.represent(file,model_name='ArcFace',detector_backend = 'retinaface',align=True,enforce_detection=True)
        except:
            pass
        try:
            arc_align = arcface_align.get_face_align(file)
            emb_arc_b[i,:] = DeepFace.represent(arc_align,model_name='ArcFace',detector_backend = 'retinaface',align=False,enforce_detection=True)
        except:
            pass

    for i in range(len(files_a)):
        for j in range(len(files_b)):
            if(np.all(emb_arc_a[i,:] == 0) or np.all(emb_dep_a[i,:] == 0) or np.all(emb_dep_b[j,:] == 0) or np.all(emb_arc_b[j,:] == 0) ):
                continue
            dis_arc = consine_sim(emb_arc_a[i,:],emb_arc_b[j,:])
            dis_dep = consine_sim(emb_dep_a[i,:],emb_dep_b[j,:])
            total_compare +=1
            filename = persona + personb + str(i) + str(j) + '.jpg'
            if(dis_arc > dis_dep + eps):
                imagea = cv2.imread(files_a[i])
                imageb = cv2.imread(files_b[j])
                os.chdir('img_comp')
                concat = np.concatenate([imagea,imageb],axis=1)
                os.chdir('./arcbetter')
                cv2.imwrite(filename,concat)
                os.chdir('..')
                os.chdir('..')
            elif(dis_dep > dis_arc + eps)
                imagea = cv2.imread(files_a[i])
                imageb = cv2.imread(files_b[j])
                os.chdir('img_comp')
                os.chdir('./depbetter')
                concat = np.concatenate([imagea,imageb],axis=1)
                cv2.imwrite(filename,concat)
                os.chdir('..')
                os.chdir('..')
            

print(total_compare,arc_more,arc_more/total_compare)     
